# Entrenar el Modelo

In [1]:
#Importar librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib #Guardar el Modelo

In [ ]:
#Cargar DataSet
print("Cargar dataset ya procesado")
df = pd.read_csv('entrenamiento.csv')
df

In [ ]:
#Asignación de variables
objetivo = 'Es_Desertor'

x = df.drop(columns=[objetivo])
y = df[objetivo]

In [ ]:
#Division de Datos 80% Entrenamiento y 20% Prueba
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
)


In [ ]:
#Entrenar el modelo
#Estimación con 100 arboles de decision
modelo = RandomForestClassifier(n_estimators=100, random_state=42)

# El modelo aprende la relación entre x e y
modelo.fit(x_train, y_train)

In [ ]:
#Guardar el modelo
joblib.dump(modelo, 'modelo_a.pkl')

In [ ]:
#Evaluar el modelo en el conjunto de prueba

#Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(x_test)

#Calcular y mostrar la precisión del modelo
print("Precisión del modelo:", accuracy_score(y_test, y_pred))

#Mostrar la matriz de confusión
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

#Mostrar el reporte de clasificación con métricas detalladas
print("Reporte de Clasificación:\n", classification_report(y_test, y_pred))  

In [ ]:
# Cargar el modelo guardado
modelo_cargado = joblib.load('modelo_a.pkl')

# Nuevo caso para predecir
nuevo_caso = {
    'Modo_solicitud': 1,
    'Orden_solicitud': 1,
    'Carrera': 1,
    'Asistencia_diurna_nocturna': 1,
    'Calificacion_previa': 1,
    'Calificacion_madre': 22,
    'Calificacion_padre': 27,
    'Ocupacion_madre': 10,
    'Ocupacion_padre': 10,
    'Desplazado': 1,
    'Deudor': 0,
    'Pagos_al_dia': 1,
    'Genero': 1,
    'Becado': 0,
    'Edad_al_matricularse': 18,
    'Unidades_1er_sem_matriculadas': 6,
    'Unidades_1er_sem_evaluaciones': 6,
    'Unidades_1er_sem_aprobadas': 6,
    'Unidades_1er_sem_nota': 14.0,
    'Unidades_2do_sem_matriculadas': 6,
    'Unidades_2do_sem_evaluaciones': 6,
    'Unidades_2do_sem_aprobadas': 6,
    'Unidades_2do_sem_nota': 13.5,
    'Tasa_desempleo': 10.8,
    'Tasa_inflacion': 1.4,
    'PIB': 1.74
}

# Convertir a DataFrame
nuevo_df = pd.DataFrame([nuevo_caso])

# Alinear columnas exactamente como el modelo
nuevo_df = nuevo_df[modelo_cargado.feature_names_in_]

# Predecir
prediccion = modelo_cargado.predict(nuevo_df)

# Resultado
if prediccion[0] == 1:
    print("El estudiante es predicho como Desertor.")
else:
    print("El estudiante es predicho como No Desertor.")


In [ ]:
prob = modelo_cargado.predict_proba(nuevo_df)

print(f"Probabilidad de No Desertor: {prob[0][0]*100:.2f}%")
print(f"Probabilidad de Desertor: {prob[0][1]*100:.2f}%")
